In [88]:
from array import array
import math

class Vector2d:
    __match_args__ = ('x', 'y') # you can see why this is needed below, we don't need to include all the public instance attributes in case of optional
    __slots__ = ('__x', '__y') # why? shown below
    typecode = 'd'

    # x, and y should be private hense two leading underscore, sometimes _ is more common but _ is sometimes used as protected and some people think protected means private :(
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))

    def __eq__(self, other):
        return tuple(self) == tuple(other)

    def __abs__(self):
        return math.hypot(self.x, self.y)

    def __bool__(self):
        return bool(abs(self))

    # added later to work with format
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'): # using p for polar coordinates
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)

    def angle(self):
        return math.atan2(self.x, self.y)

    def __hash__(self):
        return hash((self.x, self.y))

    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [14]:
v1 = Vector2d(3,4)
print(v1.x, v1.y)

3.0 4.0


In [15]:
x, y = v1
x, y

(3.0, 4.0)

In [16]:
v1

Vector2d(3.0, 4.0)

In [17]:
v1_clone = eval(repr(v1))
v1_clone

Vector2d(3.0, 4.0)

In [18]:
v1 == v1_clone

True

In [19]:
print(v1)

(3.0, 4.0)


In [20]:
octets = bytes(v1)
octets

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [21]:
abs(v1)

5.0

In [22]:
bool(v1), bool(Vector2d(0,0))

(True, False)

In [37]:
class Demo:
    @classmethod # operates on the class not instance
    def class_meth(*args): # first argument is the class itself not self
        return args

    @staticmethod
    def static_meth(*args):
        return args

In [38]:
Demo.class_meth()

(__main__.Demo,)

In [39]:
Demo.class_meth('calculus')

(__main__.Demo, 'calculus')

In [40]:
Demo.static_meth()

()

In [41]:
Demo.static_meth('calculus')

('calculus',)

In [43]:
EUR = 1/1.07 # EUR to USD coversion
EUR

0.9345794392523364

In [44]:
format(EUR, '0.4f')

'0.9346'

In [45]:
'1 EUR = {rate:0.2f} USD'.format(rate=EUR)

'1 EUR = 0.93 USD'

In [46]:
f'1 USD = {1 / EUR:0.2f} EUR'

'1 USD = 1.07 EUR'

In [47]:
format(42, 'b')

'101010'

In [48]:
format(2/3, '.1%')

'66.7%'

In [49]:
from datetime import datetime
now = datetime.now()
format(now, '%H:%M:%S')

'15:33:20'

In [50]:
"it's now {:%I:%M %p}".format(now)

"it's now 03:33 PM"

In [57]:
v1 = Vector2d(3,4)
format(v1)

'(3.0, 4.0)'

In [58]:
format(v1, '.2f')

'(3.00, 4.00)'

In [59]:
format(v1, '.3e')

'(3.000e+00, 4.000e+00)'

In [65]:
format(Vector2d(1,1),'p')

'<1.4142135623730951, 0.7853981633974483>'

In [66]:
format(Vector2d(1,1),'.3ep')

'<1.414e+00, 7.854e-01>'

In [67]:
format(Vector2d(1,1),'0.5fp')

'<1.41421, 0.78540>'

In [70]:
v1 = Vector2d(3,4)
v2 = Vector2d(3.1, 4.2)
hash(v1), hash(v2)

(1079245023883434373, 1994163070182233067)

In [71]:
{v1,v2}

{Vector2d(3.0, 4.0), Vector2d(3.1, 4.2)}

In [72]:
def keyword_pattern(v: Vector2d) -> None:
    match v:
        case Vector2d(x=0,y=0):
            print(f'{v!r} is null')
        case Vector2d(x=0):
            print(f'{v!r} is vertical')
        case Vector2d(y=0):
            print(f'{v!r} is horizontal')
        case Vector2d(x=x,y=y) if x==y:
            print(f'{v!r} is diagonal')
        case _:
            print(f'{v!r} is awesome')
# this will be fine but wont work if we add a test case:
# case Vector2d(_,0):
#     print(f'{v!r} is horizontal
# we must add __match_args__ = ('x', 'y') to our class and re-write `keyword_pattern`
def positional_pattern(v: Vector2d) -> None:
    match v:
        case Vector2d(0,0):
            print(f'{v!r} is null')
        case Vector2d(0):
            print(f'{v!r} is vertical')
        case Vector2d(_,0):
            print(f'{v!r} is horizontal')
        case Vector2d(x, y) if x==y:
            print(f'{v!r} is diagonal')
        case _:
            print(f'{v!r} iis awesome')

In [76]:
v1 = Vector2d(3,4)
v1.__dict__

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}

In [77]:
v1._Vector2d__x

3.0

In [78]:
# __slots__ uses less memory than dict, dict has significant memory overhead
# __slots__ must be present when the class is created adding or changing it later has no effect.
class Pixel:
    __slots__ = ('x', 'y')

p = Pixel()
p.__dict__ # throw

AttributeError: 'Pixel' object has no attribute '__dict__'

In [79]:
p.x = 10
p.y = 20
p.color = 'red' # throw

AttributeError: 'Pixel' object has no attribute 'color'

In [80]:
class OpenPixel(Pixel):
    pass

op = OpenPixel()
op.__dict__

{}

In [81]:
op.x = 8
op.__dict__

{}

In [82]:
op.x

8

In [83]:
op.color = 'green'
op.__dict__

{'color': 'green'}

In [84]:
# To make sure that instances of a subclass no __dict__ you must declare __slots__ again in the bubclas

In [85]:
class ColorPixel(Pixel):
    __slots__ = ('color',)

cp = ColorPixel()
cp.__dict__ # throw

AttributeError: 'ColorPixel' object has no attribute '__dict__'

In [86]:
cp.x = 2
cp.color = 'blue'
cp.flavor = 'vanilla' # throw

AttributeError: 'ColorPixel' object has no attribute 'flavor'

In [87]:
# if we use __slots__ and need __weakref__ (which is included by default in user-defined classes) we need to include it in the attributes in __slots__
# if we need to use @cached_property __dict__ must be in __slots__

In [90]:
# if we wanted a different typecode we can set it on an instance
# v1.typecode = 'f'
# if we wanted to change it on the whole class then we could simply:
# Vector2d.typecode = 'f'
# but this is silly we should be more explicit
# 'd' (8 byte double precision), 'f' (4 byte single precision float)
class ShortVector2d(Vector2d):
    typecode = 'f'

sv = ShortVector2d(1/11, 1/27)
sv

ShortVector2d(0.09090909090909091, 0.037037037037037035)

In [91]:
len(bytes(sv))

9

In [92]:
# NOTE: __slots__ are useful for millions of instances not thousands, and at that point we may want to use NumPy, pandas